In [3]:
# Importing the necessary libraries
from pathlib import Path
from datetime import datetime
import pickle
import matplotlib.pyplot as plt
import wandb

import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score

from model_architecture.DiceLoss import dice_metric_loss
from model_architecture import DUCK_Net
from image_loader import ImageLoader2D

In [3]:
# Checking the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TanLD")

NameError: name 'tf' is not defined

In [4]:
# Setting the model parameters
img_size = 352
model_type = "DuckNet"
dataset_type = 'kvasir' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
EPOCHS = 600
min_loss_for_saving = 0.2
seed_value = 58800
filters = 17 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Current timestamp for file naming
ct = datetime.now()

# Paths setup using pathlib for cross-platform compatibility
# Setting base_path to the current working directory
base_path = Path.cwd()

# Setup for directories based on the current working directory
logs_path = base_path / "logs"
models_path = base_path / "models"
results_path = base_path / "results"

# Ensure the directories exist
logs_path.mkdir(parents=True, exist_ok=True)
models_path.mkdir(parents=True, exist_ok=True)
results_path.mkdir(parents=True, exist_ok=True)

progress_csv_path = logs_path / f"{model_type}_filters_{filters}_{ct}.csv"
progress_txt_path = logs_path / f"{model_type}_filters_{filters}_{ct}.txt"
plot_path = results_path / f"{model_type}_filters_{filters}_{ct}.png"
model_save_path = models_path / f"{model_type}_filters_{filters}_{ct}.h5"

# Ensure the paths are converted to strings when used with non-Pathlib compatible functions
csv_logger = CSVLogger(str(progress_csv_path), append=True, separator=';')

NameError: name 'tf' is not defined

## EDA


In [5]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not mode None:
            images.append(img)
    return images

SyntaxError: invalid syntax (1133109215.py, line 5)

In [6]:
kvasir_path = 'datasets/Kvasir-SEG/train/'

images = load_images_from_folder(kvasir_path + 'images')
masks = load_images_from_folder(kvasir_path + 'masks')

NameError: name 'load_images_from_folder' is not defined

In [9]:
def show_images(images, masks, num=5):
    fig, axs = plt.subplots(num, 2, figsize=(10, num * 3))
    for i in range(num):
        axs[i, 0].imshow(images[i])
        axs[i, 0].set_title('Image')
        axs[i, 1].imshow(masks[i], cmap='gray')
        axs[i, 1].set_title('Mask')
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')
    plt.show()

show_images(images, masks)

NameError: name 'images' is not defined

In [ ]:
# Initialize wandb
wandb.init(project="polyp-segmentation", entity="tanld", config={
    "learning_rate": learning_rate,
    "epochs": EPOCHS,
    "img_size": img_size,
    "model_type": model_type,
    "dataset_type": dataset_type,
    "filters": filters,
    "optimizer": "RMSprop"
})


In [ ]:
# Loading the data
X, Y = ImageLoader2D.load_data(img_size, img_size, -1, 'kvasir')

In [6]:
# Splitting the data, seed for reproducibility
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle= True, random_state = seed_value)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.111, shuffle= True, random_state = seed_value)

In [7]:
# Defining the augmentations

aug_train = albu.Compose([
    albu.HorizontalFlip(),
    albu.VerticalFlip(),
    albu.ColorJitter(brightness=(0.6,1.6), contrast=0.2, saturation=0.1, hue=0.01, always_apply=True),
    albu.Affine(scale=(0.5,1.5), translate_percent=(-0.125,0.125), rotate=(-180,180), shear=(-22.5,22), always_apply=True),
])

def augment_images():
    x_train_out = []
    y_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i])
        x_train_out.append(ug['image'])  
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)

In [ ]:
# Creating the model

model = DUCK_Net.create_model(img_height=img_size, img_width=img_size, input_chanels=3, out_classes=1, starting_filters=filters)

In [19]:
# Compiling the model

model.compile(optimizer=optimizer, loss=dice_metric_loss)

In [ ]:
# Training the model

step = 0

for epoch in range(0, EPOCHS):
    
    print(f'Training, epoch {epoch}')
    print('Learning Rate: ' + str(learning_rate))

    step += 1
        
    image_augmented, mask_augmented = augment_images()
    
    csv_logger = CSVLogger(progress_path, append=True, separator=';')
    
    # Run one epoch of training
    resutl = model.fit(x=image_augmented, y=mask_augmented, epochs=1, batch_size=4, validation_data=(x_valid, y_valid), verbose=1, callbacks=[csv_logger])

    # Logging metrics to wandb
    wandb.log({
        "train_loss": result.history['loss'][0],
        "val_loss": result.history['val_loss'][0],
        "train_accuracy": result.history['accuracy'][0],
        "val_accuracy": result.history['val_accuracy'][0],
        "epoch": epoch
    })
    
    prediction_valid = model.predict(x_valid, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid).numpy()

    print("Loss Validation: " + str(loss_valid))
        
    prediction_test = model.predict(x_test, verbose=0)
    loss_test = dice_metric_loss(y_test, prediction_test)
    loss_test = loss_test.numpy()
    print("Loss Test: " + str(loss_test))
        
    with open(progressfull_path, 'a') as f:
        f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')
    
    if min_loss_for_saving > loss_valid:
        min_loss_for_saving = loss_valid
        print("Saved model with val_loss: ", loss_valid)
        model.save(model_save_path)
        wandb.run.summary["best_val_loss"] = loss_valid
        wandb.save(model_save_path)
        
    del image_augmented
    del mask_augmented

    gc.collect()

In [ ]:
# Computing the metrics and saving the results

print("Loading the model")

model = tf.keras.models.load_model(model_save_path, custom_objects={'dice_metric_loss':dice_metric_loss})

prediction_train = model.predict(x_train, batch_size=4)
prediction_valid = model.predict(x_valid, batch_size=4)
prediction_test = model.predict(x_test, batch_size=4)

print("Predictions done")

dice_train = f1_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
dice_test = f1_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
dice_valid = f1_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Dice finished")


miou_train = jaccard_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
miou_test = jaccard_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
miou_valid = jaccard_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Miou finished")


precision_train = precision_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                  np.ndarray.flatten(prediction_train > 0.5))
precision_test = precision_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
precision_valid = precision_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                  np.ndarray.flatten(prediction_valid > 0.5))

print("Precision finished")


recall_train = recall_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                            np.ndarray.flatten(prediction_train > 0.5))
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")


accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))


print("Accuracy finished")


final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('dice_train: ' + str(dice_train) + ' dice_valid: ' + str(dice_valid) + ' dice_test: ' + str(dice_test) + '\n\n')
    f.write('miou_train: ' + str(miou_train) + ' miou_valid: ' + str(miou_valid) + ' miou_test: ' + str(miou_test) + '\n\n')
    f.write('precision_train: ' + str(precision_train) + ' precision_valid: ' + str(precision_valid) + ' precision_test: ' + str(precision_test) + '\n\n')
    f.write('recall_train: ' + str(recall_train) + ' recall_valid: ' + str(recall_valid) + ' recall_test: ' + str(recall_test) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_valid: ' + str(accuracy_valid) + ' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')

In [ ]:
# Close the wandb run to clean up resources and finalize the logging session
wandb.finish()
